# Projet Final Apache Spark

**Nom Etudiant :**  

**Prenom Etudiant:**  

**Classe :**  


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Lundi 31 Janvier 2022  à 23h 59** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [4]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [5]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [6]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [7]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [8]:
fireDF.cache()

res1: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [9]:
fireDF.count()

res2: Long = 175296


In [10]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [12]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [11]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")
fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.
### Reponse 1
j'ai utilisé filter() pour filtrer les valeurs non nulles et distinct() pour eliminer les doublons.
Le nombre de types d'appels distincts qui ont été passés est 30

In [16]:

fireDF
.select("CallType")
.filter(col("CallType").isNotNull)
.distinct()
.count()

/*val distinctsCallType = data.agg(countDistinct("CallType"))
distinctsCallType.show()*/


res6: Long = 30


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [33]:
// Reponse 2
fireDF.select("CallType").distinct().show(30, false)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

 la methode "select("CallType").distinct()" permet d'afficher (Les types distincts de la colonne "CallType")

j'ai specifié 30 dans la methode .show(30, false) pour pouvoir  afficher les 30 "CallType" 


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [17]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
newFireDF
.select("ResponseDelayedinMins")
.filter(col("ResponseDelayedinMins" > 5))
.show()

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


j'ai appliqué la methode filter() sur la colonne  ResponseDelayedinMins pour recuperer

### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [23]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res12: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [48]:
/// 1) On recupere le nombre de type d'appel, pour chaque type d'appel

val CDistinctCallT =  fireDF.groupBy("CallType").count()
CDistinctCallT.orderBy("count").show(30, false)


+--------------------------------------------+------+
|CallType                                    |count |
+--------------------------------------------+------+
|Administrative                              |3     |
|Mutual Aid / Assist Outside Agency          |9     |
|Confined Space / Structure Collapse         |13    |
|Marine Fire                                 |14    |
|Suspicious Package                          |15    |
|Oil Spill                                   |21    |
|Watercraft in Distress                      |28    |
|Extrication / Entrapped (Machinery, Vehicle)|28    |
|High Angle Rescue                           |32    |
|Assist Police                               |35    |
|Aircraft Emergency                          |36    |
|Train / Rail Incident                       |57    |
|Explosion                                   |89    |
|Industrial Accidents                        |94    |
|HazMat                                      |124   |
|Fuel Spill                 

CDistinctCallT: org.apache.spark.sql.DataFrame = [CallType: string, count: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [60]:
//Reponse 5-a
// D'abord, Le nombre Zipcode
val Zipcode =  fireDF.groupBy("Zipcode").count()
Zipcode.orderBy("count").show(30,false)
fireDF.filter($"CallType" === "Medical Incident" || $"CallType" === "Structure Fire" || $"CallType" ===  "Alarms" || $"CallType" === "Traffic Collision")
    .select("Zipcode")
    .show()

+-------+-----+
|Zipcode|count|
+-------+-----+
|null   |142  |
|94129  |512  |
|94158  |882  |
|94130  |1100 |
|94104  |1341 |
|94127  |1881 |
|94111  |2974 |
|94131  |3236 |
|94123  |3719 |
|94116  |3933 |
|94108  |4084 |
|94105  |4236 |
|94132  |4321 |
|94121  |4555 |
|94134  |5009 |
|94118  |5157 |
|94114  |5175 |
|94117  |5804 |
|94133  |6246 |
|94122  |6355 |
|94107  |6941 |
|94115  |7812 |
|94112  |8421 |
|94124  |9236 |
|94109  |14686|
|94110  |14801|
|94103  |20897|
|94102  |21840|
+-------+-----+

+-------+
|Zipcode|
+-------+
|  94109|
|  94124|
|  94102|
|  94109|
|  94105|
|  94112|
|  94102|
|  94115|
|  94114|
|  94110|
|  94112|
|  94109|
|  94121|
|  94110|
|  94110|
|  94110|
|  94116|
|  94118|
|  94118|
|  94133|
+-------+
only showing top 20 rows



Zipcode: org.apache.spark.sql.DataFrame = [Zipcode: int, count: bigint]


### Question 5-b
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [65]:
// On fait un filtre avec ces deux "Zipcode" specifiquements 
fireDF .filter($"Zipcode" === 94102 || $"Zipcode" === 94103).select("Address").show()

+--------------------+
|             Address|
+--------------------+
|MARKET ST/MCALLIS...|
|600 Block of POLK ST|
|    9TH ST/HOWARD ST|
|400 Block of VALE...|
|  16TH ST/MISSION ST|
|   4TH ST/MISSION ST|
|400 Block of TURK ST|
|   OAK ST/WEBSTER ST|
| 0 Block of JONES ST|
|400 Block of EDDY ST|
|300 Block of CLEM...|
| 500 Block of OAK ST|
|700 Block of MARK...|
|HAIGHT ST/OCTAVIA ST|
|100 Block of JULI...|
|0 Block of LARKIN ST|
|100 Block of TURK ST|
|CALL BOX: BUCHANA...|
|    5TH ST/MARKET ST|
| 100 Block of 7TH ST|
+--------------------+
only showing top 20 rows



### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [66]:
//Reponse 6
fireDF.groupBy("CallType").count().show(31,false)

+--------------------------------------------+------+
|CallType                                    |count |
+--------------------------------------------+------+
|Elevator / Escalator Rescue                 |453   |
|Marine Fire                                 |14    |
|Aircraft Emergency                          |36    |
|Confined Space / Structure Collapse         |13    |
|Administrative                              |3     |
|Alarms                                      |19406 |
|Odor (Strange / Unknown)                    |490   |
|Citizen Assist / Service Call               |2524  |
|HazMat                                      |124   |
|Watercraft in Distress                      |28    |
|Explosion                                   |89    |
|Oil Spill                                   |21    |
|Vehicle Fire                                |854   |
|Suspicious Package                          |15    |
|Extrication / Entrapped (Machinery, Vehicle)|28    |
|Other                      

In [20]:
newFireDF
.select("ResponseDelayedinMins").describe()
.show()

+-------+---------------------+
|summary|ResponseDelayedinMins|
+-------+---------------------+
|  count|               175296|
|   mean|    3.892364154521585|
| stddev|    9.378286226254257|
|    min|          0.016666668|
|    max|              1844.55|
+-------+---------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [35]:

fireTSDF.select(year(col("IncidentDate")))
.distinct()
.count()

res21: Long = 19


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [37]:
// Calculer le nombre d'incident par semaine en 2018
fireTSDF.groupBy($"week_of_year",$"IncidentDate")
.agg(count($"IncidentDate").as("count"))
.filter($"IncidentDate"
.between("2018-01-01 00:00:00","2018-12-31 00:00:00"))
.orderBy(desc("count"))
.show(10)

org.apache.spark.sql.AnalysisException:  cannot resolve '`week_of_year`' given input columns: [ALSUnit, Address, AvailableDtTS, Battalion, Box, CallFinalDisposition, CallNumber, CallType, CallTypeGroup, City, FinalPriority, FirePreventionDistrict, IncidentDate, IncidentNumber, Location, Neighborhood, NumAlarms, OnWatchDate, OriginalPriority, Priority, ResponseDelayedinMins, RowID, StationArea, SupervisorDistrict, UnitID, UnitSequenceInCallDispatch, UnitType, Zipcode];;

### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [54]:
fireTSDF
  .select("Neighborhood", "ResponseDelayedinMins") where(year($"IncidentDate") == 2018)
.show(10, false)

<console>: 39: error: value show is not a member of Boolean

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [ ]:
//Reponse 9

fireDF
.write.format("parquet")
.saveAsTable(parquetTable)
df6.write.parquet("MySparkproject.parquet")


### Question 10
**Comment relire les données stockée en format Parquet?**

In [ ]:
val parquetFileDF = spark.read.parquet("MySparkproject.parquet")

## FIN